<a href="https://colab.research.google.com/github/okana2ki/intro-to-DS/blob/main/DS13_2v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `SSDSE-C-2024.csv`の読み込み

In [ ]:
import pandas as pd

url = "https://www.nstac.go.jp/sys/files/SSDSE-C-2024.csv"
df = pd.read_csv(url, header=1, encoding="shift_jis")
print(df)

## AIへのデータ解説

**この後、AIにプログラム作成を依頼したときに、このデータを適切に解釈してもらえるように、↓下のセルで、コメント行の形で、このデータの形式と内容を（AIに対して）説明しておく。←これは重要です。これがあるのとないのとでは、適切なプログラムが生成される可能性が大きく変わるはず。**試しに、下記のセルよりも上のセルでAIに依頼して比べてみて下さい（AIは依頼したセルよりも下は読み込まないので）。（もちろん、プログラム作成を依頼するときに、このような情報を付けて依頼することもできる。）

↓下のセル内の
説明は、[サイト](https://www.nstac.go.jp/use/literacy/ssdse/) に掲載されている[データ解説PDFファイル](https://www.nstac.go.jp/sys/files/kaisetsu-C-2024.pdf) に基づいて書きました。（AIだけでなく）皆さんも、この解説PDFに目を通してからデータ分析するようにして下さい。

`header=1`と指定したため、変数`df`は、元のCSVファイルから、1行目を削除した内容になっていることに注意して下さい。

In [ ]:
# dfデータの解説
# データのレイアウト
# 1行目は項目名、それ以下の48行がデータ。データ行は、「全国」と「47都道府県庁所在市」に対応。
# 列は、最初の3列が地域情報を示す。1行目から順に「地域コード」「都道府県」「市」を示す。
# 4列目以降がデータで、4列目から順に「世帯人員」「食料（合計）」と続くが、5列目は、食品（合計）に対する1世帯当たり年間支出金額で、
# 6列目以降は、食品の品目ごとの1世帯当たり年間支出金額のデータである。
# なお、次の12個の列は、それぞれ、それ以降の列の小計になっている：「穀類」「魚介類」「肉類」「乳卵類」「野菜・海藻」「果物」「油脂・調味料」「菓子類」「調理食品」「飲料」「酒類」「外食」

## グラフ中の日本語表示を可能にする

AIに依頼すると、グラフ中に日本語表示するプログラムを作成することがあるため、グラフ中に日本語が表示できるように、あらかじめ、下記の2つのコードセルを実行しておきましょう。

In [ ]:
!pip install japanize_matplotlib

In [3]:
import japanize_matplotlib

# データのランキング

ランキングとは、順位付けです。たとえば、宮崎は餃子への支出額が一位とか、そういうやつです。



In [ ]:
# 数値データ列だけを対象に(numeric_only=True)、列ごとに降順でランキング
df_rank = df.rank(numeric_only=True, ascending=False)
print(df_rank)

In [ ]:
df_city = df[['市']]  # 上記のランキングには市の列が含まれないので、別途、市の列を抽出
print(df_city)

In [ ]:
# 市のDataFrameとランキングのDataFrameを連結→市の列を含むランキングが完成
df_city_rank = pd.concat([df_city, df_rank], axis=1)
print(df_city_rank)

In [ ]:
# prompt: df_city_rankから、宮崎市の行を抜き出して下さい。

# 宮崎市の行を抽出
df_miyazaki_rank = df_city_rank[df_city_rank['市'] == '宮崎市']

# 結果を表示
df_miyazaki_rank

In [ ]:
# 省略されて全部の列が見れないので、全部表示されるようにするために、今、何列まで表示される設定になっているか調べる
print(pd.options.display.max_columns)
# これで調べると、データ部分が20列まで省略されずに表示されるようになっていることが分かる。

In [ ]:
# そこで、227カラム全部表示されるように設定する。一度表示したら、元の20に戻しておく。
pd.set_option('display.max_columns', 227)
print(df_miyazaki_rank)
pd.set_option('display.max_columns', 20)

In [ ]:
df_miyazaki_rank_t = df_miyazaki_rank.T
# .Tによる転置（行と列の入れ替え）←後の処理の都合で転置しておいた方が便利そうなので
print(df_miyazaki_rank_t)

In [ ]:
df_miyazaki_rank_t[df_miyazaki_rank_t.iloc[1:,:] <= 15]
# 15位以内を表示；interactive tableに変換すると全部の行が見える

In [ ]:
df_miyazaki_rank_t[df_miyazaki_rank_t.iloc[1:,:] >= 46]
# 46位以降（少ない方から3位以内）を表示；interactive tableに変換すると全部の行が見える

In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib  # 日本語対応（グラフ中で日本語が表示できるようにする）

df_miyazaki_rank_t.drop(df_miyazaki_rank_t.index[[0]]).plot.bar(figsize=(40,5))
# 数値データでない0行目を削除(.drop)して棒グラフを表示(.plot.bar)

## 演習問題
以上の結果から、宮崎は食料品消費額ランキング上位の品目は少なく、消費額ランキング下位の品目が多いことが分かりますね。この傾向は、食料（合計）のランキングが43位（下から6番目）であることにも表れています。

宮崎は物価が安いから消費額が少ないのか、年収が少ない人が多いから少ないのか、それぞれがどれくらい効いて、このような結果になっているのでしょうか？
[エンゲル係数](https://www.rakuten-card.co.jp/minna-money/topic/article_2103_00007/)の話は、これを考察する上で参考になりそうですね。
いずれにせよ、このことをちゃんと分析・考察するには、物価のデータや、年収のデータを合わせて調べる必要があります。
興味がある人は調べてみて下さい。

## 演習問題
**品目ごとの消費額を食料品全体の消費額（食料（合計））で割り算して、割合でランキング**すると、宮崎の特徴を表す別の傾向が見えそうです。ぜひ分析してみて下さい。地域ごとの物価の違いの影響をある程度抑えて、それぞれの市がどのような消費傾向を持っているかが見えるのではないかと思います。ただし、「宮崎ではピーマンが安い」のような個々の品目の価格の違いの影響は残ったままです。宮崎では全体として食品の物価が安いという傾向（があるとしたら、それ）は打ち消せるのですが。都市ごとの年収の違いの影響も抑えられるかな・・・

以下は、割合でランキングするプログラムです。

In [ ]:
import pandas as pd

file_name = "/content/SSDSE-C-2023.csv"
df = pd.read_csv(file_name, header=[0,1], encoding="shift_jis")
print(df)

In [ ]:
dfr = df  # 割合で置き換えたデータを格納するDataFrame dfrを用意
for column_name, item in dfr.iloc[:,5:229].items():  # 5列目から228列目までを割合に書き換えるループ
  dfr[column_name] = dfr[column_name] / dfr[('LB00','食料（合計）')]
print(dfr)

In [ ]:
# 数値データ列だけを対象に(numeric_only=True)、列ごとに降順でランキング
dfr_rank = dfr.rank(numeric_only=True, ascending=False)
print(dfr_rank)

In [ ]:
dfr_miyazaki_rank = dfr_rank[45:46]  # 45行の宮崎市を抽出
pd.set_option('display.max_columns', 227)  # 全部の列を表示
print(dfr_miyazaki_rank)
pd.set_option('display.max_columns', 20)

In [ ]:
dfr_miyazaki_rank_t = dfr_miyazaki_rank.T
# .Tによる転置（行と列の入れ替え）←後の処理の都合で転置しておいた方が便利そうなので
print(dfr_miyazaki_rank_t)

In [19]:
dfr_miyazaki_rank_t[dfr_miyazaki_rank_t.iloc[1:,:] <= 15]
# 15位以内を表示；interactive tableに変換すると全部の行が見える

NameError: name 'dfr_miyazaki_rank_t' is not defined

In [18]:
dfr_miyazaki_rank_t[dfr_miyazaki_rank_t.iloc[1:,:] >= 46]
# 46位以降（少ない方から3位以内）を表示；interactive tableに変換すると全部の行が見える

NameError: name 'dfr_miyazaki_rank_t' is not defined

In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib  # 日本語対応（グラフ中で日本語が表示できるようにする）

dfr_miyazaki_rank_t.plot.bar(figsize=(40,5))
# 棒グラフを表示(.plot.bar)

# ヒストグラム
チューハイ・カクテルへの支出金額のヒストグラム

In [ ]:
import pandas as pd
import matplotlib.pylab as plt

file_name = "/content/SSDSE-C-2023.csv"
df = pd.read_csv(file_name, header=[0,1], encoding="shift_jis")
# print(df)

print(df.filter(like='チューハイ', axis=1))  # ヘッダーに'チューハイ'を含む列を検索

plt.hist(df[('LB110007','チューハイ・カクテル')])

# 散布図
りんご×オレンジの散布図

In [ ]:
!pip install japanize-matplotlib  # 日本語対応（グラフ中で日本語が表示できるようにする）

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib  # 日本語対応（グラフ中で日本語が表示できるようにする）

file_name = "/content/SSDSE-C-2023.csv"
df = pd.read_csv(file_name, header=[0,1], encoding="shift_jis")
# print(df)

# print(df.filter(like='りんご', axis=1))  # ヘッダーに'りんご'を含む列を検索
# print(df.filter(like='オレンジ', axis=1))  # ヘッダーに'オレンジ'を含む列を検索

x = df[('LB061001','りんご')]
y = df[('LB061004','オレンジ')]
city = df[('City','市')]
# print(city)

plt.figure(figsize=(15, 12))
plt.scatter(x,y) # 散布図
for i in range(len(x)): # 各点に市のラベルを付ける
    # 文字が重なってしまう都道府県名の表示位置を調整
    if city[i] in ['神戸市', '千葉市', '徳島市', '長崎市', '鹿児島市', '高松市', '宇都宮市', '名古屋市']:
        plt.text(x[i], y[i], city[i], ha='left', va='bottom')
    else:
        plt.text(x[i], y[i], city[i], ha='right', va='bottom')

plt.xlabel('りんごへの支出額')
plt.ylabel('オレンジへの支出額')
plt.show()

# df_city_ringo_orange = df[[('City','市'),('LB061001','りんご'),('LB061004','オレンジ')]]  # 市の列とりんごの列とオレンジの列を抽出
# print(df_city_ringo_orange)

# 参考
以下は、「データのランキング」のセクション後半で、支出額を割合で上書きするプログラムの作成時に、その動作を小さいDataFrameで確認したときのもの。参考までに掲載します。

なお、本日の小テストはこの（下記の）プログラムに関する問題です。**わらかないときは、生成AIに尋ねると教えてくれるはずですが、答を尋ねるのでなく、プログラムの分からない点を質問したりする方をお勧めします。その方が力がつくので。答をすぐに教えることを避ける設定にしてあると、そのように対応してくれるかもしれません。**

In [ ]:
import pandas as pd

dfs = pd.DataFrame({'a': [1,2], 'b': [3,4], 'c': [5,6]})
print(dfs)

for column_name, item in dfs.iloc[:,1:3].items():
  dfs[column_name] = dfs[column_name] / dfs['a']
print(dfs)